# Manejo de valores nulos y atípicos

In [1]:
import sys
import os
import pandas as pd

current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.eda_analyzer import EDAAnalyzer
from scripts.transform.imputer_handler import ImputerHandler

In [2]:
df = pd.read_csv('../../data/raw/Estudiantes_clean.csv')
eda = EDAAnalyzer()
imputer = ImputerHandler()

In [3]:
eda.plot_null_heatmap(df)

Eliminar la columna 'demuestra_confianza' porque tiene muy pocos valores llenos

In [4]:
df = df.drop(columns=['demuestra_confianza'])

## Analizar variables categóricas

In [5]:
cols_categoricas = ['estrato', 'tipo_vivienda', 'interés_estudios_superiores', 'familia', 'actividades_extracurriculares', 'proyección_vocacional']

### Estrato

In [6]:
eda.plot_barplots(df, ['estrato'])

In [7]:
df_imputed = imputer.impute_missing_values(df, 'estrato', metodo="moda")

### Tipo de vivienda

In [8]:
eda.plot_barplots(df, ['tipo_vivienda'])

In [9]:
df_imputed = imputer.impute_missing_values(df_imputed, 'tipo_vivienda', metodo="moda")

### Interés en estudios superiores

In [10]:
eda.plot_barplots(df, ['interés_estudios_superiores'])

In [11]:
df_imputed = imputer.impute_missing_values(df_imputed, 'interés_estudios_superiores', metodo="moda")

### Composición familiar

In [12]:
eda.plot_barplots(df, ['familia'])

In [13]:
df_imputed = imputer.impute_missing_values(df_imputed, 'familia', metodo="moda")

### Actividades extracurriculares

In [14]:
eda.plot_barplots(df, ['actividades_extracurriculares'])

In [15]:
df_imputed = imputer.impute_missing_values(df_imputed, 'actividades_extracurriculares', metodo="moda")

### Proyección vocacional

In [16]:
eda.plot_barplots(df, ['proyección_vocacional'])

In [17]:
df_imputed = imputer.impute_missing_values(df_imputed, 'proyección_vocacional', metodo="moda")

## Analizar variables numéricas

In [18]:
variables_numericas = ['horas_semana_estudio_casa', 'total_hermanos']

In [19]:
# Estadísticas descriptivas
df_imputed[variables_numericas].describe()


,horas_semana_estudio_casa,total_hermanos
count,278.000000,278.000000
mean,2.096196,1.241007
std,5.401752,1.086266
min,0.000000,0.000000
25%,0.000000,1.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,68.000000,8.000000


In [20]:
eda.plot_histograms(df_imputed, variables_numericas)

In [21]:
eda.plot_boxplots(df_imputed, variables_numericas)

### Horas de estudio en casa

In [22]:
# Llenamos los valores nulos con la media
df_imputed['horas_semana_estudio_casa'] = df_imputed['horas_semana_estudio_casa'].fillna(df_imputed['horas_semana_estudio_casa'].mean())

### Número de hermanos

In [23]:
# Llenamos los valores nulos con la mediana
df_imputed['total_hermanos'] = df_imputed['total_hermanos'].fillna(df_imputed['total_hermanos'].median())

## Manejo de valores atípicos

In [24]:
cols_to_search = [col for col in df.columns if col not in ['ID','sede', 'nivel', 'grado', 'orden_grado', 'año_ingreso', 'género', 'documento_identificación', 'primer_apellido', 'segundo_apellido', 'nombres', 'fecha_nacimiento', 'dirección', 'valoración_emocional']]
cols_to_search

['antigüedad',
 'país_origen',
 'estrato',
 'tipo_vivienda',
 'zona_vivienda',
 'horas_semana_estudio_casa',
 'interés_estudios_superiores',
 'medio_transporte',
 'apoyo_familiar',
 'total_hermanos',
 'familia',
 'actividades_extracurriculares',
 'enfermedades',
 'proyección_vocacional',
 'participación_clase',
 'nee',
 'nivel_motivación']

In [25]:
variables_numericas = ['horas_semana_estudio_casa', 'total_hermanos']

In [26]:
eda.detect_outliers(df_imputed, variables_numericas)


horas_semana_estudio_casa - Atípicos detectados: 16



total_hermanos - Atípicos detectados: 143


{'horas_semana_estudio_casa': 12      7.0
 57     30.0
 61      8.0
 75      9.0
 77      6.0
 185    30.0
 195    68.0
 224    20.0
 228    30.0
 242     6.0
 268     6.0
 271     7.0
 273     8.0
 283     7.0
 292    10.0
 359    10.0
 Name: horas_semana_estudio_casa, dtype: float64,
 'total_hermanos': 1      0.0
 2      0.0
 5      3.0
 6      0.0
 7      4.0
       ... 
 353    2.0
 354    2.0
 360    0.0
 361    0.0
 362    0.0
 Name: total_hermanos, Length: 143, dtype: float64}

In [27]:
# Manejo de atípicos horas_semana_estudio_casa
# Llenamos los valores atípicos con la mediana
df_imputed = imputer.handle_outliers(df_imputed, 'horas_semana_estudio_casa', "mediana")
eda.plot_boxplots(df_imputed, ['horas_semana_estudio_casa'])

In [28]:
variables_categoricas = list(set(cols_to_search) - set(variables_numericas))
variables_categoricas

['país_origen',
 'zona_vivienda',
 'actividades_extracurriculares',
 'proyección_vocacional',
 'enfermedades',
 'interés_estudios_superiores',
 'medio_transporte',
 'nee',
 'familia',
 'antigüedad',
 'tipo_vivienda',
 'apoyo_familiar',
 'estrato',
 'nivel_motivación',
 'participación_clase']

In [29]:
eda.detect_rare_categories(df_imputed, variables_categoricas)

país_origen

país_origen - Categorías raras (< 5 apariciones):
['Argentina', 'España', 'República dominicana']
zona_vivienda

zona_vivienda - Categorías raras (< 5 apariciones):
[]
actividades_extracurriculares

actividades_extracurriculares - Categorías raras (< 5 apariciones):
['Artes, Idiomas', 'Artes, Deporte, Idiomas', 'Tecnología / Diseño']
proyección_vocacional

proyección_vocacional - Categorías raras (< 5 apariciones):
[]
enfermedades

enfermedades - Categorías raras (< 5 apariciones):
[]
interés_estudios_superiores

interés_estudios_superiores - Categorías raras (< 5 apariciones):
[]
medio_transporte

medio_transporte - Categorías raras (< 5 apariciones):
['Bicicleta']
nee

nee - Categorías raras (< 5 apariciones):
[]
familia

familia - Categorías raras (< 5 apariciones):
[]
antigüedad

antigüedad - Categorías raras (< 5 apariciones):
[]
tipo_vivienda

tipo_vivienda - Categorías raras (< 5 apariciones):
[]
apoyo_familiar

apoyo_familiar - Categorías raras (< 5 apariciones):
[

{'país_origen': ['Argentina', 'España', 'República dominicana'],
 'zona_vivienda': [],
 'actividades_extracurriculares': ['Artes, Idiomas',
  'Artes, Deporte, Idiomas',
  'Tecnología / Diseño'],
 'proyección_vocacional': [],
 'enfermedades': [],
 'interés_estudios_superiores': [],
 'medio_transporte': ['Bicicleta'],
 'nee': [],
 'familia': [],
 'antigüedad': [],
 'tipo_vivienda': [],
 'apoyo_familiar': [],
 'estrato': [],
 'nivel_motivación': [],
 'participación_clase': []}

In [31]:
eda.plot_null_heatmap(df_imputed)

In [32]:
# Guardamos el dataframe limpio
df_imputed.to_csv('../../data/raw/Estudiantes_imputed.csv', index=False)